In [2]:
## standard modules to import
import pandas as pd
import re
from nameparser import HumanName

# standard pandas display options
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', -1)


In [5]:
# Use a lookup table to assign SFDC account ID based on domain

def return_domain(email):
    # takes an email address and returns a simplified domain (without email etc.)
    try:
        domain = re.split("@", email)[1]
        simplified_domain = re.split("\.", domain)[0]
        return domain
    except:
        return ""


# load domain lookup list and de_dupe
# note that we really need to sort out this list - there are a lot of incorrectly assigned "email domain" values
domain_list = pd.read_csv('domain to account id.csv')
domain_list = domain_list.drop_duplicates(['Account Email Domain'])

lead_list['Domain'] = lead_list.apply(lambda row: return_domain(row['Email']),axis=1)

lead_list = lead_list.merge(domain_list, 
                left_on='Domain', 
                right_on='Account Email Domain', 
                how='left')

In [ ]:
# Convert country info into Salesforce format
lead_list['Country'] = lead_list['Country'].replace(
    {'USA':'United States', 
     'KOREA':'Korea, Republic of',
     'THE NETHERLANDS':'Netherlands',
     'RUSSIA':'Russian Federation'}) 

# case insensitive replacement would be better but I couldn't figure it out


In [ ]:
# Change abbreviated states to long form

# load state table csv
states = pd.read_csv("states_table.csv")

# left join with states table for full names
lead_list = lead_list.merge(states[['Abbreviation', 'State']],
                            left_on='State_abbrev',
                            right_on='Abbreviation',
                            how='left')

# delete unnecessary columns
lead_list = lead_list.drop(['Abbreviation'], axis=1)


In [ ]:
# (sometimes) convert a full name into first and last names and remove pre/post nominals

def first_name(unformatted_name):
    name = HumanName(unformatted_name)
    name.capitalize(force=True)
    return name.first

def last_name(unformatted_name):
    name = HumanName(unformatted_name)
    name.capitalize(force=True)
    return name.last


In [4]:
# assign seniority based on title

def seniority_assignment(df):
    # takes df with 'Title' column, returns df with seniority column matching SFDC conventions
    
    SENIOR = ['Senior', 'Sr', 'manager', 'mgr', 'lead', 'head', 'principle', 'principal']
    DIRECTOR = ['Director', 'Dir']
    VP = [' vice', 'President', 'VP', 'vice president']
    EXECUTIVE = ['chief', 'Executive', 'CEO', 'CIO', 'CSO', 'CTO ', 'founder']

    # create list of regex search patterns
    seniority_levels = []
    for level in [SENIOR, DIRECTOR, VP, EXECUTIVE]:
        esc_lst = [re.escape(s) for s in level]
        pattern = '|'.join(esc_lst)
        seniority_levels.append(pattern)

    # assign seniority level based on keywords
    df.loc[df.Title.str.contains(seniority_levels[0], na=False, case=False, regex=True), 'Seniority'] = 'Senior'
    df.loc[df.Title.str.contains(seniority_levels[1], na=False, case=False, regex=True), 'Seniority'] = 'Director'
    df.loc[df.Title.str.contains(seniority_levels[2], na=False, case=False, regex=True), 'Seniority'] = 'VP'
    df.loc[df.Title.str.contains(seniority_levels[3], na=False, case=False, regex=True), 'Seniority'] = 'Executive'

    # default is 'Entry'
    df.Seniority = df.Seniority.fillna(value = 'Entry')

    # add CTO - this was picking up direCTOr before I added a space to the search term
    df.loc[df.Title == 'CTO', 'Seniority'] = 'Executive'
    
    return df
